<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

In [1]:
# Parameters
msgs = "Ran from Airflow at 2022-03-20T17:37:49.786591+00:00!"


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
import os
import scipy
import numpy as np
import pandas as pd
from pathlib import Path
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from sklearn import set_config
set_config(display = 'diagram')
from pandas.api.types import infer_dtype


# Scikit Learn import
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.pipeline import FeatureUnion
# from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn import cluster
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputRegressor


# Category Encoder
import category_encoders as ce
# pyod
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.cof import COF
from pyod.models.iforest import IForest
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA as PCA_pyod
from pyod.models.mcd import MCD
from pyod.models.sod import SOD
from pyod.models.sos import SOS
# model
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from imblearn.over_sampling import SMOTE

import re

import vnquant.DataLoader as dl
from datetime import datetime, timedelta
import pytz
from pathlib import Path


ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
init_date = '2020-01-01'
start_date = '2021-01-01'
end_date = '2022-03-20'
symbols = ["HSG", "HPG", "NKG"]
def get_data(symbol, start_date = start_date, end_date = end_date):
    loader = dl.DataLoader(symbols=[symbol], start=start_date, end=end_date, minimal=True)
    data = loader.download()
    data.columns = [col[0] for col in data.columns]
    data = data.reset_index()
    data['symbol'] = symbol
    data['target_day_1'] = data.close.shift(-1)
    data['target_day_2'] = data.close.shift(-2)
    data['target_day_3'] = data.close.shift(-3)
    data['target_day_4'] = data.close.shift(-4)
    data['target_day_5'] = data.close.shift(-5)
    return data
df = pd.concat([get_data(symbol) for symbol in symbols], axis = 0)
df = df.sort_values(by=['date'])
df.tail(10)

In [ ]:
numeric_columns = ["high", "low", "open", "close", "volume"]
category_columns = ['symbol']
time_columns = ['date']
feature_columns = numeric_columns+category_columns+time_columns
target_columns = ['target_day_1', 'target_day_2', 'target_day_3', 'target_day_4', 'target_day_5']
# X = df.loc[df[target_columns].isnull().any(axis = 1)].drop(columns=target_columns)
# y = df.loc[df[target_columns].notnull().any(axis = 1)][target_columns]
# test = df.loc[df[target_columns].isnull().any(axis = 1)]

In [ ]:
class TimePreprocess(BaseEstimator, TransformerMixin):
    def __init__(self, list_of_features=["day", "dayofweek", "month"],):
        super().__init__()
        self.time_features = []
        self.list_of_features = list_of_features
    def fit(self, X, y = None):
        for col in X.columns:
            try:
                X[col] = pd.to_datetime(X[col])
                self.time_features.append(col)
            except:
                logger.error('Time columns cannot be convert to datetime')
        return self
    def transform(self, X):
        columns = []
        for col in self.time_features:
            X[col] = pd.to_datetime(X[col])
            if 'day' in self.list_of_features:
                columns.append(col+'_day')
                X[columns[-1]] = X[col].dt.day
            if 'month' in self.list_of_features:
                columns.append(col+'_month')
                X[columns[-1]] = X[col].dt.month
            if 'dayofweek' in self.list_of_features:
                columns.append(col+'_dayofweek')
                X[columns[-1]] = X[col].dt.dayofweek
        return X[columns]
    def fit_transform(self, X, y = None):
        self.fit(X, y)
        return self.transform(X)


numeric_preprocess  = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler_quantile', QuantileTransformer()),
    # ('scaler', MinMaxScaler())
])
category_preprocess = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', ce.OneHotEncoder(use_cat_names = True)),
])

In [ ]:


preprocessing = ColumnTransformer(transformers=[
        ('numeric_preprocess', numeric_preprocess, numeric_columns),
        ('category_columns', category_preprocess, category_columns),
        ('time_preprocess', TimePreprocess(), time_columns),
        # ('hold_feature', HoldFeature(), hold_columns),
    ],
                                      n_jobs = -1
                                     )

pipe = Pipeline(steps = [
    ('preprocessing', preprocessing),
    ('estimator', CatBoostRegressor())
])
# pipe.fit(X, y)
params = {
    'estimator': [CatBoostRegressor(verbose = 0),
                  # LGBMRegressor(verbose = 0),
                  # XGBRegressor()
                 ]
}
grid =  GridSearchCV(estimator = pipe, 
                     param_grid = params,
                     scoring = 'neg_mean_squared_error',
                     cv = TimeSeriesSplit(n_splits=5),
                     n_jobs = -1
                    )
for target_col in target_columns:
    X = df.loc[df[target_col].notnull(), feature_columns]
    y = df.loc[df[target_col].notnull(), target_col]
    grid.fit(X, y)
    print(grid.cv_results_)
    break

In [ ]:
grid.cv_results_

In [ ]:
X_test['target'] = grid.best_estimator_.predict(X_test)

In [ ]:
X_test